In [1]:
import torch
from torch import nn
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

# 3.7.1. 初始化模型参数

In [2]:
# PyTorch不会隐式地调整输入的形状。因此，
# 我们在线性层前定义了展平层（flatten），来调整网络输入的形状
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 10)
)

# 自动对模型net的各层进行递归访问，并对其中的Linear线性层进行参数初始化
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

# 3.7.2. 重新审视Softmax的实现
交叉熵损失。从计算角度来看，指数可能会造成数值稳定性问题。
解决这个问题的一个技巧是： 在继续softmax计算之前，先从所有$o_k$中减去$\max(o_k)$。

这里可以看到每个$o_k$按常数进行的移动不会改变softmax的返回值：
$$\[\begin{split}\begin{aligned} \hat y_j & = \frac{\exp(o_j - \max(o_k))\exp(\max(o_k))}{\sum_k \exp(o_k - \max(o_k))\exp(\max(o_k))} \\ & = \frac{\exp(o_j - \max(o_k))}{\sum_k \exp(o_k - \max(o_k))}. \end{aligned}\end{split}\]$$

在减法和规范化步骤之后，可能有些$o_j - \max(o_k)$具有较大的负值。 由于精度受限，$\exp(o_j - \max(o_k))$将有接近零的值，即下溢（underflow）。 这些值可能会四舍五入为零，使$\hat y_j$为零， 并且使得$\log(\hat y_j)$的值为$-\infty$。 反向传播几步后，我们可能会发现自己面对一屏幕可怕的nan结果。

通过将softmax和交叉熵结合在一起，可以避免反向传播过程中可能会困扰我们的数值稳定性问题。
$$\[\begin{split}\begin{aligned} \log{(\hat y_j)} & = \log\left( \frac{\exp(o_j - \max(o_k))}{\sum_k \exp(o_k - \max(o_k))}\right) \\ & = \log{(\exp(o_j - \max(o_k)))}-\log{\left( \sum_k \exp(o_k - \max(o_k)) \right)} \\ & = o_j - \max(o_k) -\log{\left( \sum_k \exp(o_k - \max(o_k)) \right)}. \end{aligned}\end{split}\]$$

In [3]:
loss = nn.CrossEntropyLoss(reduction='none')

# 3.7.3. 优化算法

In [4]:
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

# 3.7.4. 训练

In [5]:
num_epochs = 10
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

AttributeError: module 'd2l.torch' has no attribute 'train_ch3'